<a href="https://colab.research.google.com/github/RajasreePushpan/yes_no_questions/blob/main/yes_no_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -U

In [21]:
import pandas as pd

In [64]:
data = pd.read_csv("/content/True or False questions - Sheet1 (1).csv",on_bad_lines='skip', engine="python")
data.head()

,Number,Question,Potential Answer,Correct Answer,Ground Truth,Location,Type
0,1,Statistics involves the science of collecting ...,True/False,True,"Statistics is the science of collecting, organ...",Page 3,True/False
1,2,Organizing data is not a part of the statistic...,True/False,False,"Statistics is the science of collecting, organ...",Page 3,True/False
2,3,Presenting data is an important aspect of stat...,True/False,True,"Statistics is the science of collecting, organ...",Page 3,True/False
3,4,Analyzing data is not necessary in statistics.,True/False,False,"Statistics is the science of collecting, organ...",Page 3,True/False
4,5,"Descriptive statistics help in organizing, sum...",True/False,True,Descriptive statistics include methods of orga...,page 4,True/False


In [65]:
# Convert the 'Correct Answer' column to string type, clean by stripping spaces, and convert to lowercase
data['Correct Answer'] = data['Correct Answer'].astype(str).str.strip().str.lower()

# Map 'true'/'false' to 1/0
data['Correct Answer'] = data['Correct Answer'].map({'true': 1, 'false': 0})

# Check for NaN values to debug
print(data['Correct Answer'].isna().sum())

0


In [ ]:
print(data.head())

In [66]:
# Save the modified dataframe to a new CSV file (optional)
data.to_csv("/content/True_or_False_questions_modified.csv", index=False)

In [67]:
data = data[['Question','Correct Answer']]
data = data[0:250]
data.head()

,Question,Correct Answer
0,Statistics involves the science of collecting ...,1
1,Organizing data is not a part of the statistic...,0
2,Presenting data is an important aspect of stat...,1
3,Analyzing data is not necessary in statistics.,0
4,"Descriptive statistics help in organizing, sum...",1


In [68]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification


In [69]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

model

In [71]:
model = model.to('cuda')

In [72]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [73]:
X = list(data["Question"])
y = list(data["Correct Answer"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [74]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [75]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [76]:
len(X_train),len(X_val)

(200, 50)

In [77]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [78]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [79]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [84]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=20,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [85]:

trainer.train()

Step,Training Loss
500,0.021400


TrainOutput(global_step=500, training_loss=0.021421566009521484, metrics={'train_runtime': 66.8299, 'train_samples_per_second': 59.853, 'train_steps_per_second': 7.482, 'total_flos': 78111094560000.0, 'train_loss': 0.021421566009521484, 'epoch': 20.0})

In [86]:

trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 3.1643197536468506,
 'eval_accuracy': 0.66,
 'eval_precision': 0.631578947368421,
 'eval_recall': 0.5454545454545454,
 'eval_f1': 0.5853658536585366,
 'eval_runtime': 0.2081,
 'eval_samples_per_second': 240.317,
 'eval_steps_per_second': 33.644,
 'epoch': 20.0}

In [49]:
np.set_printoptions(suppress=True)

In [87]:
text = "That was good point"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2820,  0.2335]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.3739, 0.6261]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.3739001, 0.6260999]], dtype=float32)

In [88]:

trainer.save_model('CustomModel')

In [ ]:
# file ipython-input-52-9d5e9197470e

model_2 = BertForSequenceClassification.from_pretrained("/content/CustomModel")
model_2.to('cuda')

In [56]:
# text = "That was good point"
text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.5369113 , 0.46308872]], dtype=float32)